## Problem Set 4

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset4_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing

In [19]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# reduce traceback
%xmode Minimal

Exception reporting mode: Minimal


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset4_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [20]:
# check where we are
os.getcwd()

# load data

debar = pd.read_csv("/Users/maggiesullivan/Documents/DS_1/PS/PS4/debar.csv")

'/Users/maggiesullivan/Documents/DS_1/ppol564_f22_assignments2/code/PS4_submission'

## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [21]:
# check the number of rows
debar.info()   # yields 105 rows (plus useful references)
debar.shape    # also yields 105 rows

# check number of unique employer names
debar.Name.nunique()   # yields 94 unique names

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         105 non-null    object
 1   City, State  104 non-null    object
 2   Violation    105 non-null    object
 3   Start date   105 non-null    object
 4   viol_num     105 non-null    object
dtypes: object(5)
memory usage: 4.2+ KB


(105, 5)

94

**Response:** There are multiple rows for some employers since there are 105 rows but only 94 unique names. 

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [22]:
## A
# Create a new column in debar--is_repeated-- that tells us whether an employer (Name) is repeated > 1 times

# group by Name column and find duplicates
name_count = debar.groupby('Name').size()
name_count

# subset to where the name count is > 1
multiple = name_count.index[name_count > 1]
multiple

# create a new column with this
debar['is_repeated'] = debar.Name.isin(multiple)

# check one we know is doubled 
debar.loc[debar.Name == "Xavier Horne"]

Name
69 Farms LLC              1
AB Ranch                  1
Abe- Nancy Froese         1
Agecy I LLC               1
Agecy II, LLC             1
                         ..
Walker Place              1
Xavier Horne              2
Yesenia Perez             1
Yolanda Chavez            1
Yolanda Chavez Farming    1
Length: 94, dtype: int64

Index(['Autumn Hill Orchard', 'Cisco Produce Inc.', 'Dove Creek Farms',
       'F&W Farms', 'Macky and Brad Farms', 'Mark Duncan',
       'Old Tree Farms/Verpaalen Custom Service',
       'Rollo Farm Labor Contractor', 'SRT Farms', 'Sharon Mathis',
       'Xavier Horne'],
      dtype='object', name='Name')

,Name,"City, State",Violation,Start date,viol_num,is_repeated
54,Xavier Horne,"Lyons, Georgia",Non-payment of certification fee,6/16/2016,viol1,True
63,Xavier Horne,"Lyons, Georgia",Failure to respond to audit request,9/27/2017,viol2,True


In [23]:
## B

# Print the rows where is_repeated == True and interpret
repeated = debar.loc[debar.is_repeated == True].reset_index().copy()
repeated

# Sort them by name to see if there are any key differences or patterns
repeated.sort_values(by=['Name'])

# We see that many of the repeated names have violations with the same start date and slight variations 
# in the violation name. 
# For example: 
repeated.loc[repeated.Name == "Dove Creek Farms"]

,index,Name,"City, State",Violation,Start date,viol_num,is_repeated
0,0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,True
1,9,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,7/6/2014,viol2,True
2,14,Macky and Brad Farms,"Plains, TX",Failure to respond to audit (no response),2/13/2015,viol2,True
3,23,SRT Farms,"Morton, TX",Failure to respond to audit (no response),11/16/2014,viol1,True
4,24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,True
5,26,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",WHD Debarment,12/11/2014,viol1,True
6,37,Rollo Farm Labor Contractor,"Miami, FL",Impeding the Audit Process – Non- Response,8/23/2014,viol2,True
7,44,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),8/23/2014,viol1,True
8,48,F&W Farms,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1,True
9,49,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,12/10/2015,viol1,True


,index,Name,"City, State",Violation,Start date,viol_num,is_repeated
1,9,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,7/6/2014,viol2,True
18,95,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),7/6/2014,viol1,True
13,70,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),12/10/2014,viol2,True
9,49,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,12/10/2015,viol1,True
21,102,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2/9/2018,viol2,True
4,24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,True
15,74,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),12/10/2014,viol2,True
8,48,F&W Farms,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1,True
0,0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,True
2,14,Macky and Brad Farms,"Plains, TX",Failure to respond to audit (no response),2/13/2015,viol2,True


,index,Name,"City, State",Violation,Start date,viol_num,is_repeated
4,24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,True
21,102,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2/9/2018,viol2,True


In [24]:
## C

# Subset to the rows where is_repeated == True and save that data as mult_debar. 
mult_debar = debar.loc[debar.is_repeated == True].reset_index().copy()
 
# Print the head() and shape
mult_debar.head()
mult_debar.shape

,index,Name,"City, State",Violation,Start date,viol_num,is_repeated
0,0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,True
1,9,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,7/6/2014,viol2,True
2,14,Macky and Brad Farms,"Plains, TX",Failure to respond to audit (no response),2/13/2015,viol2,True
3,23,SRT Farms,"Morton, TX",Failure to respond to audit (no response),11/16/2014,viol1,True
4,24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,True


(22, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [25]:
# pivot from long to wide
mult_debar_wide = pd.pivot(mult_debar[['Name', 'City, State', 'viol_num', 'Violation',
                                      'Start date']], 
                          index = ['Name', 'City, State'],
                          columns = ['viol_num'],
                          values = ["Start date"]).copy()

# check to see this worked
mult_debar_wide

# rename columns
mult_debar_wide.columns = ['start_date_viol1', 'start_date_viol2']

# print head and shape
mult_debar_wide.head()
mult_debar_wide.shape

Start date  \
viol_num                                                       viol1   
Name                                    City, State                    
Autumn Hill Orchard                     Groton, MA          7/6/2014   
Cisco Produce Inc.                      Cairo, GA         12/10/2015   
Dove Creek Farms                        Mount Vernon, TX    2/9/2018   
F&W Farms                               Ingalls, KS       12/10/2014   
Macky and Brad Farms                    Plains, TX         2/13/2015   
Mark Duncan                             Roosevelt, UT     11/16/2014   
Old Tree Farms/Verpaalen Custom Service Volga, SD         12/11/2014   
Rollo Farm Labor Contractor             Miami, FL          8/23/2014   
SRT Farms                               Morton, TX        11/16/2014   
Sharon Mathis                           Tifton, GA        11/16/2014   
Xavier Horne                            Lyons, Georgia     6/16/2016   

                                                                      
viol_num                                                       viol2  
Name                                    City, State                   
Autumn Hill Orchard                     Groton, MA          7/6/2014  
Cisco Produce Inc.                      Cairo, GA         12/10/2014  
Dove Creek Farms                        Mount Vernon, TX    2/9/2018  
F&W Farms                               Ingalls, KS       12/10/2014  
Macky and Brad Farms                    Plains, TX         2/13/2015  
Mark Duncan                             Roosevelt, UT     11/16/2014  
Old Tree Farms/Verpaalen Custom Service Volga, SD          12/1/2014  
Rollo Farm Labor Contractor             Miami, FL          8/23/2014  
SRT Farms                               Morton, TX        11/16/2014  
Sharon Mathis                           Tifton, GA        11/16/2014  
Xavier Horne                            Lyons, Georgia     9/27/2017

,,start_date_viol1,start_date_viol2
Name,"City, State",,
Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
Cisco Produce Inc.,"Cairo, GA",12/10/2015,12/10/2014
Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015


(11, 2)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`

    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [26]:
## A

# Using mult_debar_wide, add a column is_dup that takes value of True for cases where 
    #start_date_viol1 == start_date_viol2 marking the row as a duplicate

dup = mult_debar_wide.start_date_viol1 == mult_debar_wide.start_date_viol2
dup
    
mult_debar_wide['is_dup'] = dup == True
mult_debar_wide

Name                                     City, State     
Autumn Hill Orchard                      Groton, MA           True
Cisco Produce Inc.                       Cairo, GA           False
Dove Creek Farms                         Mount Vernon, TX     True
F&W Farms                                Ingalls, KS          True
Macky and Brad Farms                     Plains, TX           True
Mark Duncan                              Roosevelt, UT        True
Old Tree Farms/Verpaalen Custom Service  Volga, SD           False
Rollo Farm Labor Contractor              Miami, FL            True
SRT Farms                                Morton, TX           True
Sharon Mathis                            Tifton, GA           True
Xavier Horne                             Lyons, Georgia      False
dtype: bool

,,start_date_viol1,start_date_viol2,is_dup
Name,"City, State",,,
Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
Cisco Produce Inc.,"Cairo, GA",12/10/2015,12/10/2014,False
Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True
Old Tree Farms/Verpaalen Custom Service,"Volga, SD",12/11/2014,12/1/2014,False
Rollo Farm Labor Contractor,"Miami, FL",8/23/2014,8/23/2014,True
SRT Farms,"Morton, TX",11/16/2014,11/16/2014,True


In [27]:
# B

# First subset wide data to only names with  
dup = mult_debar_wide[mult_debar_wide.is_dup == True].reset_index()
dup_names = dup.Name
dup_names

# Then use this to subset the debar into two dfs

# one keeps only viol1 of duplicate data
debar_keepv1 = debar[(debar.Name.isin(dup_names)) & (debar.viol_num == "viol1")].copy()

# the other keeps all the other names and values
debar_keepallv = debar[~debar.Name.isin(dup_names)].copy()

# now lets join them together
debar_concat = pd.concat([debar_keepv1, debar_keepallv]).copy()

# check to see this worked
debar_concat.shape

# Remove the is_repeated column from the debar data
debar_clean = debar_concat.drop('is_repeated', axis = 1)
debar_clean.head()
debar_clean.shape

0            Autumn Hill Orchard
1               Dove Creek Farms
2                      F&W Farms
3           Macky and Brad Farms
4                    Mark Duncan
5    Rollo Farm Labor Contractor
6                      SRT Farms
7                  Sharon Mathis
Name: Name, dtype: object

(97, 6)

,Name,"City, State",Violation,Start date,viol_num
0,Macky and Brad Farms,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1
23,SRT Farms,"Morton, TX",Failure to respond to audit (no response),11/16/2014,viol1
24,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1
44,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),8/23/2014,viol1
48,F&W Farms,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1


(97, 5)

# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [28]:
# load datset
jobs = pd.read_csv("/Users/maggiesullivan/Documents/DS_1/PS/PS4/jobs_clean.csv")
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2720 entries, 0 to 2719
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CASE_NUMBER         2720 non-null   object
 1   EMPLOYER_NAME       2720 non-null   object
 2   EMPLOYER_CITY       2720 non-null   object
 3   EMPLOYER_STATE      2720 non-null   object
 4   EMPLOYER_ADDRESS_1  2720 non-null   object
dtypes: object(5)
memory usage: 106.4+ KB


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [29]:
## A

# Use pd.merge with an inner join on those fields to see whether there are any exact matches
inner_merge = pd.merge(jobs, debar_clean,
                      how = "inner",
                      left_on = "EMPLOYER_NAME",
                      right_on = 'Name')

# print to see if this worked 
inner_merge

## B this inner merge yielded only one match

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,Name,"City, State",Violation,Start date,viol_num
0,H-300-20287-876656,Rafael Barajas,Port St. Lucie,FL,2598 SW Dalpina Rd.,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [30]:
## A 

# Convert the EMPLOYER_NAME and Name fields to uppercase using list comprehension rather than df.varname.str.upper() 
name_upper = [name.upper() for name in debar_clean.Name]
employer_upper = [name.upper() for name in jobs.EMPLOYER_NAME]

len(employer_upper)
len(name_upper)

2720

97

In [31]:
## B

# print a random sample of each
random.sample(employer_upper, 15)
random.sample(name_upper, 15)

['HIRSCH VINEYARDS, LLC',
 'EVERGLADES HARVESTING, INC.',
 'GREEN HILL FARMS',
 'H2A COMPLETE II, INC.',
 'GREEN DIRT FARM LLC',
 'WALLACE FARMS A PARTNERSHIP',
 'AGRICO LABOR LLC',
 'CASTLEPARK FARM LLC',
 '7K RANCH INC.',
 'KYLE R. FONTENOT',
 'WESTERN RANGE ASSOCIATION',
 'LAWSON FARMS PARTNERSHIP',
 'COASTAL FARM LABOR SERVICES, INC.',
 'MATTHEW PERRON',
 'GULF SOUTH GROWERS ASSOCIATION, INC.']

['LESLIE COOK',
 'JIM AND ANN SHIPLEY WILLIAM SHIPLEY*',
 'E.V. RANCH LLP',
 'F&W FARMS',
 'CHRIS ROBINSON',
 'YOLANDA CHAVEZ FARMING',
 'J&J HARVESTING',
 'AUTUMN HILL ORCHARD',
 'GREAT PLAINS FLUID SERVICE, INC.',
 'TURNER FARMS',
 'ANTON FERTILIZER INC.',
 'XAVIER HORNE',
 'MACKY AND BRAD FARMS',
 'ALTERIC JEAN-CHARLES',
 'CISCO PRODUCE INC.']

In [32]:
## C

# Assign the full vector of uppercase names back to the original data, 
    # writing over the original EMPLOYER_NAME and Name columns

jobs.EMPLOYER_NAME = employer_upper
debar_clean.Name = name_upper

# check this worked
jobs.head()
debar_clean.head()

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1
0,H-300-20199-721302,"FAZIO FARMS OPERATING COMPANY, LLC",Portland,OR,8433 NE 13th Ave
1,H-300-20231-773906,CHARLIE SUNDERLAND,McMinnville,TN,982 Northcutts Cove Road
2,H-300-20231-774123,MICHAEL RUDEBUSCH,Livingston,TX,7366 State Hwy 146 S
3,H-300-20231-774151,LODAHL FARMS,Brockton,MT,4198 Road 2049
4,H-300-20231-774508,"DUNSON HARVESTING, INC.",Winter Haven,FL,400 Eagle Lake Loop Rd.


,Name,"City, State",Violation,Start date,viol_num
0,MACKY AND BRAD FARMS,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1
23,SRT FARMS,"Morton, TX",Failure to respond to audit (no response),11/16/2014,viol1
24,DOVE CREEK FARMS,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1
44,ROLLO FARM LABOR CONTRACTOR,"Miami, FL",Failure to respond to audit (no response),8/23/2014,viol1
48,F&W FARMS,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1


### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [33]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [34]:
## A

# for jobs
# first test for one issue (INC)
period_pattern_1 = r"(INC\.)"
works = [re.sub(period_pattern_1, "INC", name) for name in jobs.EMPLOYER_NAME]
#works

# then apply for multiple
period_pattern_all = r"(INC|CO|LLC)\."
replacement_all = r"\1"
jobs_period_clean = [re.sub(period_pattern_all, replacement_all, name) for name in jobs.EMPLOYER_NAME]

## do the same for debar_clean
debar_period_clean = [re.sub(period_pattern_all, replacement_all, name) for name in debar_clean.Name]

In [35]:
## B 

# apply this pattern for each of the examples
re.sub(period_pattern_all, replacement_all, pos_example_1)
re.sub(period_pattern_all, replacement_all, pos_example_2)
re.sub(period_pattern_all, replacement_all, neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [36]:
## clean the columns and add to each df
jobs["name_clean"] = jobs_period_clean 
debar_clean["name_clean"] = debar_period_clean

In [37]:
# print head of each df
jobs[['EMPLOYER_NAME', "name_clean"]].head()
debar_clean[["Name","name_clean"]].head()

,EMPLOYER_NAME,name_clean
0,"FAZIO FARMS OPERATING COMPANY, LLC","FAZIO FARMS OPERATING COMPANY, LLC"
1,CHARLIE SUNDERLAND,CHARLIE SUNDERLAND
2,MICHAEL RUDEBUSCH,MICHAEL RUDEBUSCH
3,LODAHL FARMS,LODAHL FARMS
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"


,Name,name_clean
0,MACKY AND BRAD FARMS,MACKY AND BRAD FARMS
23,SRT FARMS,SRT FARMS
24,DOVE CREEK FARMS,DOVE CREEK FARMS
44,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR
48,F&W FARMS,F&W FARMS


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate the fuzzy matching we'll cover in future week or optional OH; that fuzzy matching would recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical

In [38]:
## A 

# Conduct another inner join between jobs and debar_clean now using the name_clean column
inner_merge_2 = pd.merge(jobs, debar_clean,
                      how = "inner",
                      left_on = "EMPLOYER_NAME",
                      right_on = 'Name')

# print the result to see if the cleaning resulted in any more employers matched between the two datasets
inner_merge_2    # this still matches only one employer

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,name_clean_x,Name,"City, State",Violation,Start date,viol_num,name_clean_y
0,H-300-20287-876656,RAFAEL BARAJAS,Port St. Lucie,FL,2598 SW Dalpina Rd.,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1,RAFAEL BARAJAS


In [39]:
## B

# Use regex to take SLASH E.V. RANCH LLP in the debar_clean dataset to match
    # SLASH EV RANCH in the jobs dataset
    # need to remove the periods AND the space+LLP 
slash_pattern = r"(E|V)\.|(\sLLP)"
slash_replacement = r"\1" 
#test_on_one = re.sub(slash_pattern, slash_replacement,"SLASH E.V. RANCH LLP")
#test_on_one    # this works
slash_clean = [re.sub(slash_pattern, slash_replacement, name) for name in debar_clean.Name]
#slash_clean

# Create a new column in debar_clean called name_clean_2
    # you can apply the pattern to all employer names in debar_clean 
    # aka so don't need to worry about only applying it to that one employer
debar_clean['name_clean_2'] = slash_clean
debar_clean.head()

,Name,"City, State",Violation,Start date,viol_num,name_clean,name_clean_2
0,MACKY AND BRAD FARMS,"Plains, TX",Impeding the Audit Process – Partial- Response,2/13/2015,viol1,MACKY AND BRAD FARMS,MACKY AND BRAD FARMS
23,SRT FARMS,"Morton, TX",Failure to respond to audit (no response),11/16/2014,viol1,SRT FARMS,SRT FARMS
24,DOVE CREEK FARMS,"Mount Vernon, TX",Failure to Respond to Audit Request,2/9/2018,viol1,DOVE CREEK FARMS,DOVE CREEK FARMS
44,ROLLO FARM LABOR CONTRACTOR,"Miami, FL",Failure to respond to audit (no response),8/23/2014,viol1,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR
48,F&W FARMS,"Ingalls, KS",Impeding the Audit Process – Partial- Response,12/10/2014,viol1,F&W FARMS,F&W FARMS


In [40]:
## C

# Conduct a left join using name_clean_2 as the join column where the left hand dataframe is jobs; 
    # right hand dataframe is debar_clean, store the result as a dataframe, 
left_joined_employers = pd.merge(jobs, debar_clean,
                                 how = "left",
                                 left_on = 'EMPLOYER_NAME',
                                 right_on = "name_clean_2",
                                 indicator = "name_merge_status")

# print the rows where the merge indicator indicates the row was found in both dataframe
matches = left_joined_employers[left_joined_employers.name_merge_status != "left_only"]
matches

,CASE_NUMBER,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_ADDRESS_1,name_clean_x,Name,"City, State",Violation,Start date,viol_num,name_clean_y,name_clean_2,name_merge_status
791,H-300-20287-876656,RAFAEL BARAJAS,Port St. Lucie,FL,2598 SW Dalpina Rd.,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,9/23/2016,viol1,RAFAEL BARAJAS,RAFAEL BARAJAS,both
1115,H-300-20306-894148,SLASH EV RANCH,Rifle,CO,22593 County Road 5,SLASH EV RANCH,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,11/15/2014,viol1,SLASH E.V. RANCH LLP,SLASH EV RANCH,both
